# 10 — ALOS‑2 Seasonal Composites (L‑band)
Compute wet/dry γ0 dB composites for ALOS‑2 PALSAR‑2 ScanSAR (Level 2.2) over a Brazilian AOI,
then produce Δ = wet − dry maps and list the contributing scenes (IDs) for evidence logging.

**Dataset:** `JAXA/ALOS/PALSAR-2/Level2_2/ScanSAR` (GEE). Conversion DN→γ0 dB: `10*log10(DN^2) - 83`.
Docs: https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR-2_Level2_2_ScanSAR


In [ ]:
import json, datetime as dt
import ee
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# ---- Parameters ----
params = {
    'aoi_bbox': [-57.0, -3.0, -54.0, -1.0],  # [min_lon, min_lat, max_lon, max_lat] — TODO: update
    'start': '2015-01-01',
    'end': '2025-01-01',
    'wet_months': [12,1,2,3,4,5],
    'dry_months': [6,7,8,9,10,11],
    'pol': 'HH',  # 'HH' or 'HV'
    'scale': 30,
}
geom = ee.Geometry.Rectangle(params['aoi_bbox'])

def to_gamma0db(img):
    pol = params['pol']
    dn = img.select([pol])
    gamma0 = dn.pow(2).log10().multiply(10).subtract(83).rename(f'{pol}_db')
    return img.addBands(gamma0, overwrite=True)

col = (
    ee.ImageCollection('JAXA/ALOS/PALSAR-2/Level2_2/ScanSAR')
    .filterBounds(geom)
    .filterDate(params['start'], params['end'])
    .map(to_gamma0db)
)

def month_filter(months):
    mset = ee.List(months)
    return col.filter(ee.Filter.inList('month', mset))

# Add month property for filtering
col = col.map(lambda i: i.set('month', ee.Date(i.get('system:time_start')).get('month')))

wet = month_filter(params['wet_months']).median().select([f"{params['pol']}_db"]).rename('wet_db')
dry = month_filter(params['dry_months']).median().select([f"{params['pol']}_db"]).rename('dry_db')
delta = wet.subtract(dry).rename('delta_db')

# Collect contributing scene IDs
ids_wet = month_filter(params['wet_months']).aggregate_array('system:index').getInfo()
ids_dry = month_filter(params['dry_months']).aggregate_array('system:index').getInfo()
print('Wet scenes:', len(ids_wet))
print('Dry scenes:', len(ids_dry))

# Export params (for logging)
meta = {
    'aoi_bbox': params['aoi_bbox'],
    'pol': params['pol'],
    'wet_months': params['wet_months'],
    'dry_months': params['dry_months'],
    'ids_wet': ids_wet,
    'ids_dry': ids_dry,
}
print(json.dumps(meta, indent=2)[:1000])


### Visualization (optional; requires geemap)

In [ ]:
try:
    import geemap
    m = geemap.Map()
    m.centerObject(geom, 8)
    viz = {'min': -20, 'max': 5}
    m.add_layer(wet, viz, 'wet_db')
    m.add_layer(dry, viz, 'dry_db')
    m.add_layer(delta, {'min': -5, 'max': 5}, 'delta_db')
    m.addLayerControl()
    m
except Exception as e:
    print('Install geemap to visualize:', e)


### Evidence logging (JSONL)
Uses the repo utility if available (`src/zexplorer`).

In [ ]:
try:
    import sys
    sys.path.append('src')
    from zexplorer.data_id_logger import log_evidence, DataSource
    rec = log_evidence(
        lat=(params['aoi_bbox'][1]+params['aoi_bbox'][3])/2,
        lon=(params['aoi_bbox'][0]+params['aoi_bbox'][2])/2,
        candidate_id='rat-delta-0001',
        sources=[
            DataSource(type='ALOS2 ScanSAR L2.2 (wet)', id=str(ids_wet[:5]) + '...'),
            DataSource(type='ALOS2 ScanSAR L2.2 (dry)', id=str(ids_dry[:5]) + '...'),
        ],
        bbox=params['aoi_bbox'],
        notes='ΔdB seasonal composite over AOI; see notebook 10.',
    )
    print('Logged evidence line.')
except Exception as e:
    print('Skipping logging (ensure src/ path available):', e)
